<a href="https://colab.research.google.com/github/netra-poonia/Incentive-Calculator-Colab-notebooks/blob/main/StateHead_Incentive_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## `State Business Head Incentive Calculatoin`

In [1]:
class IncentiveCalculator:
    """
    Calculates incentives for State Business Heads based on performance metrics.

    Metrics considered:
    - Disbursement and AUM Growth (75% weight)
    - Productivity (Median Secured Disbursals per RM, 25% weight)
    - Portfolio Quality (Current % and Early Delinquency %, adjustment factor)
    """

    def __init__(self, disbursement_aum_growth_percentage, median_secured_disbursals_per_rm, current_percentage, early_delinquency_percentage):
        """
        Initializes the IncentiveCalculator with performance metrics for a State Business Head.

        Args:
            disbursement_aum_growth_percentage: The percentage growth in disbursement and AUM
                                                relative to the target (e.g., 110 for 110% achievement).
            median_secured_disbursals_per_rm: The median secured disbursals per Relationship Manager
                                              in a given period.
            current_percentage: The percentage of the portfolio that is in current status.
            early_delinquency_percentage: The percentage of the portfolio that is in early delinquency.
        """
        self.disbursement_aum_growth_percentage = disbursement_aum_growth_percentage
        self.median_secured_disbursals_per_rm = median_secured_disbursals_per_rm
        self.current_percentage = current_percentage
        self.early_delinquency_percentage = early_delinquency_percentage

    def calculate_disbursement_aum_incentive(self):
        """
        Calculates the incentive based on the percentage of target achievement for Disbursement and AUM Growth.

        Assumes sample base and bonus payout values for different achievement slabs.
        The slabs and payouts are hardcoded within the method for simplicity.

        Returns:
            The calculated incentive amount for disbursement and AUM growth.
        """
        # Define slabs and corresponding base and bonus payouts for Disbursement and AUM Growth
        disbursement_aum_slabs = {
            (0, 80): {'base': 0, 'bonus': 0}, # Below 80% achievement
            (80, 100): {'base': 50000, 'bonus': 0}, # 80% to 99.99% achievement
            (100, 120): {'base': 50000, 'bonus': 20000}, # 100% to 119.99% achievement
            (120, float('inf')): {'base': 50000, 'bonus': 50000} # 120% and above achievement
        }

        incentive = 0
        # Iterate through the defined slabs to find the applicable one
        for (lower_bound, upper_bound), payouts in disbursement_aum_slabs.items():
            if lower_bound <= self.disbursement_aum_growth_percentage < upper_bound:
                incentive = payouts['base'] + payouts['bonus']
                break # Found the correct slab, exit the loop

        return incentive

    def calculate_productivity_incentive(self):
        """
        Calculates the incentive based on the median secured disbursals per RM.

        Assumes sample payout values for different ranges of median secured disbursals.
        The ranges and payouts are hardcoded within the method.

        Returns:
            The calculated productivity incentive amount.
        """
        # Define ranges and corresponding payout values for Productivity
        productivity_buckets = {
            (0, 500000): 0, # Below 500,000 disbursals per RM
            (500000, 1000000): 20000, # 500,000 to 999,999 disbursals per RM
            (1000000, 1500000): 40000, # 1,000,000 to 1,499,999 disbursals per RM
            (1500000, float('inf')): 60000 # 1,500,000 and above disbursals per RM
        }

        incentive = 0
        # Iterate through the defined buckets to find the applicable one
        for (lower_bound, upper_bound), payout in productivity_buckets.items():
            if lower_bound <= self.median_secured_disbursals_per_rm < upper_bound:
                incentive = payout
                break # Found the correct bucket, exit the loop

        return incentive

    def calculate_portfolio_quality_adjustment(self):
        """
        Calculates the adjustment factor based on the Current % and Early Delinquency %.

        Assumes sample adjustment factor values based on a predefined matrix.
        The matrix is hardcoded within the method.

        Returns:
            The calculated portfolio quality adjustment factor (e.g., 0.8, 1.0, 1.2).
        """
        # Sample portfolio quality adjustment matrix:
        # Keys are tuples representing (Current % lower bound, Current % upper bound)
        # Values are dictionaries where keys are tuples representing (Early Delinquency % lower bound, Early Delinquency % upper bound)
        # and values are the adjustment factors.
        portfolio_quality_matrix = {
            (0, 80): { # Current % is below 80%
                (0, 2): 0.8, # Early Delinquency below 2%
                (2, 5): 0.6, # Early Delinquency 2% to 4.99%
                (5, float('inf')): 0.4 # Early Delinquency 5% and above
            },
            (80, 90): { # Current % is 80% to 89.99%
                (0, 2): 1.0, # Early Delinquency below 2%
                (2, 5): 0.8, # Early Delinquency 2% to 4.99%
                (5, float('inf')): 0.6 # Early Delinquency 5% and above
            },
            (90, float('inf')): { # Current % is 90% and above
                (0, 2): 1.2, # Early Delinquency below 2%
                (2, 5): 1.0, # Early Delinquency 2% to 4.99%
                (5, float('inf')): 0.8 # Early Delinquency 5% and above
            }
        }

        adjustment_factor = 1.0  # Default to no adjustment if no matching bucket is found (should not happen with comprehensive buckets)

        # Iterate through the Current % buckets
        for (current_lower, current_upper), delinquency_buckets in portfolio_quality_matrix.items():
            if current_lower <= self.current_percentage < current_upper:
                # If Current % matches a bucket, iterate through the Early Delinquency % buckets
                for (delinquency_lower, delinquency_upper), factor in delinquency_buckets.items():
                    if delinquency_lower <= self.early_delinquency_percentage < delinquency_upper:
                        adjustment_factor = factor
                        break  # Found the delinquency bucket, break the inner loop
                break  # Found the current percentage bucket, break the outer loop

        return adjustment_factor

    def calculate_final_incentive(self):
        """
        Calculates the final incentive by combining weighted incentives from Disbursement/AUM and Productivity
        and applying the portfolio quality adjustment factor.

        Disbursement and AUM Growth has a 75% weight.
        Productivity has a 25% weight.
        The sum of weighted incentives is multiplied by the portfolio quality adjustment factor.

        Returns:
            The final calculated incentive amount.
        """
        # Calculate incentives for each component
        disbursement_aum_incentive = self.calculate_disbursement_aum_incentive()
        productivity_incentive = self.calculate_productivity_incentive()
        portfolio_quality_adjustment = self.calculate_portfolio_quality_adjustment()

        # Calculate the weighted sum of Disbursement/AUM and Productivity incentives
        weighted_incentive = (disbursement_aum_incentive * 0.75) + (productivity_incentive * 0.25)

        # Apply the portfolio quality adjustment factor to the weighted incentive
        final_incentive = weighted_incentive * portfolio_quality_adjustment

        return final_incentive

def process_bulk_incentives(data):
    """
    Processes a bulk dataset of State Business Heads to calculate incentives.

    Iterates through a list of performance data dictionaries, creates an
    IncentiveCalculator instance for each, and calculates the final incentive.

    Args:
        data: A list of dictionaries, where each dictionary contains performance metrics
              for a State Business Head with keys matching the IncentiveCalculator
              __init__ arguments ('Disbursement and AUM Growth %',
              'Median Secured Disbursals per RM', 'Current %', 'Early Delinquency %').

    Returns:
        A list of calculated final incentives (float) for each business head in the
        same order as the input data.
    """
    results = []
    # Process each business head's data in the list
    for business_head_data in data:
        # Create an instance of the IncentiveCalculator with the current business head's data
        calculator = IncentiveCalculator(
            business_head_data['Disbursement and AUM Growth %'],
            business_head_data['Median Secured Disbursals per RM'],
            business_head_data['Current %'],
            business_head_data['Early Delinquency %']
        )
        # Calculate the final incentive for the current business head
        final_incentive = calculator.calculate_final_incentive()
        # Append the calculated incentive to the results list
        results.append(final_incentive)
    return results

def process_single_incentive(disbursement_aum_growth_percentage, median_secured_disbursals_per_rm, current_percentage, early_delinquency_percentage):
    """
    Calculates the incentive for a single State Business Head.

    Creates an IncentiveCalculator instance with the provided performance metrics
    and calculates the final incentive.

    Args:
        disbursement_aum_growth_percentage: Percentage growth in disbursement and AUM.
        median_secured_disbursals_per_rm: Median secured disbursals per Relationship Manager.
        current_percentage: Percentage of current portfolio.
        early_delinquency_percentage: Percentage of early delinquency portfolio.

    Returns:
        The final calculated incentive (float) for the single business head.
    """
    # Create an instance of the IncentiveCalculator with the single business head's data
    calculator = IncentiveCalculator(
        disbursement_aum_growth_percentage,
        median_secured_disbursals_per_rm,
        current_percentage,
        early_delinquency_percentage
    )
    # Calculate the final incentive
    final_incentive = calculator.calculate_final_incentive()
    return final_incentive

## `Final Calculation`

In [2]:
bulk_data = [
    {
        'Disbursement and AUM Growth %': 110,
        'Median Secured Disbursals per RM': 1200000,
        'Current %': 85,
        'Early Delinquency %': 1.5
    },
    {
        'Disbursement and AUM Growth %': 95,
        'Median Secured Disbursals per RM': 750000,
        'Current %': 92,
        'Early Delinquency %': 3.0
    },
    {
        'Disbursement and AUM Growth %': 130,
        'Median Secured Disbursals per RM': 1600000,
        'Current %': 78,
        'Early Delinquency %': 6.0
    }
]

# Sample data for one-on-one processing
single_data = {
    'Disbursement and AUM Growth %': 105,
    'Median Secured Disbursals per RM': 900000,
    'Current %': 88,
    'Early Delinquency %': 2.5
}

# Process bulk data and print results
print("Bulk Processing Results:")
bulk_results = process_bulk_incentives(bulk_data)
for i, result in enumerate(bulk_results):
    print(f"Business Head {i+1} Final Incentive: {result}")

# Process single data and print result
print("\nOne-on-One Processing Result:")
single_result = process_single_incentive(
    single_data['Disbursement and AUM Growth %'],
    single_data['Median Secured Disbursals per RM'],
    single_data['Current %'],
    single_data['Early Delinquency %']
)
print(f"Single Business Head Final Incentive: {single_result}")

Bulk Processing Results:
Business Head 1 Final Incentive: 62500.0
Business Head 2 Final Incentive: 42500.0
Business Head 3 Final Incentive: 36000.0

One-on-One Processing Result:
Single Business Head Final Incentive: 46000.0
